# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

# Описание данных
<br> Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц.
<br> Описание столбцов:
- `сalls` — количество звонков
- `minutes`— суммарная длительность звонков в минутах
- `messages` — количество sms-сообщений
- `mb_used` — израсходованный интернет-трафик в Мб
- `is_ultra` — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0)

## Откройте и изучите файл

In [1]:
# Импортируем библиотеки
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [2]:
# открываем файл
df = pd.read_csv('/datasets/users_behavior.csv')

In [3]:
# выведем основную информацию о файле
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


Таблица состоит из 3214 строк и 5ти столбцов, пропуски отсутствуют.

In [4]:
# изучаем первые 10 строк
df.head(10)

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0


In [5]:
# выведем основные параметры
df.describe()

,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


**Вывод**
<br> С данными ознакомились, они готовы для дальнейшей работы

## Разбейте данные на выборки

Спрятанной тестовой выборки нет, Значит данные будем разбивать на 3 части:
- обучающую (60%)
- тестовую (20%)
- валидационную (20%)

In [6]:
# разделим данные на 2 набора: обучающий и тестовый, присвоив размер тестового 40%
train, test = train_test_split(df, test_size=0.4, random_state=12345)

In [7]:
features = test.drop('is_ultra', axis=1) # выделим в данных признаки
target = test['is_ultra'] # выделим из данных целевой признак - столбец с тарифом

In [8]:
# разделим тестовый набор данных, полученный ранее, еще на 2 набора: вылидационный и тестовый, 50% от 40% присвоенных ранее (т.е. будет по 20% для выборок)
features_val, features_test, target_val, target_test = train_test_split(
    features, target, test_size=0.5, random_state=12345)


In [9]:
features_train = train.drop('is_ultra', axis=1)  # выделим в обучающих данных признаки
target_train = train['is_ultra'] # выделим из обучающих данных целевой признак

In [10]:
# проверим размеры данных
features_train.shape, features_test.shape, features_val.shape 

((1928, 4), (643, 4), (643, 4))

In [11]:
# проверим распределение данных по выборкам в %
print(f"train = {100*len(train)/len(df):.1f}%")
print(f"val = {100*len(features_val)/len(df):.1f}%")
print(f"test = {100*len(features_test)/len(df):.1f}%")

train = 60.0%
val = 20.0%
test = 20.0%


**Вывод**
<br> разбили данные на следующие выборки:
- features_train, target_train - обучающая выборка - 60% данных
- features_test, target_test - тестовая выборка - 20% данных
- features_val, target_val - валидационная выборка - 20% данных

## Исследуйте модели

### Дерево решений

In [12]:
best_result = 0 # счетчик точности модели
best_depth = 0 # счетчик глубины дерева
# создадим цикл для max_depth от 1 до 10
for depth in range(1,10):
    model = DecisionTreeClassifier(max_depth=depth, splitter='best', random_state=12345, min_samples_split=2) # создаем объект дерева решений
    model.fit(features_train, target_train) # обучим модель на тренировочной выборке
    result = model.score(features_val, target_val) # получим предсказания модели на валидационной выборке
    if result > best_result:
        best_result = result
        best_depth = depth
print("Глубина дерева:", best_depth, " Точность модели:", best_result)


Глубина дерева: 3  Точность модели: 0.7853810264385692


### Случайный лес

In [13]:
# проверим модель на алгоритме случайного леса (параметры глубины дерева от 1 до 10, леса: кол-во деревьев от 10 до 50 с шагом 10)
best_result = 0
best_depth = 0
for depth in range(1,10):
    for est in range(10,50,10):
        model = RandomForestClassifier(max_depth=depth, n_estimators=est, random_state=12345)
        model.fit(features_train, target_train)
        result = model.score(features_val, target_val)
        if result > best_result:
            best_result = result
            best_depth = depth
            best_est = est
print("Глубина дерева:", best_depth, " Количество деревьев:", best_est, " Качество:", best_result)


Глубина дерева: 8  Количество деревьев: 40  Качество: 0.8087091757387247


### Логистическая регрессия

In [14]:
# проверим модель алгоритмом логистической регрессии
model = LogisticRegression(random_state=12345, solver='liblinear') # инициализируем модель логистической регрессии с параметром random_state=12345
model.fit(features_train, target_train) # обучим модель на тренировочной выборке
result = model.score(features_val, target_val) # получим метрику качества модели на валидационной выборке

print("Точность модели:", result)


Точность модели: 0.7589424572317263


### Вывод

Наибольшая точность модели на валидационной выборке достигается при применении алгоритма RandomForestClassifier (случайный лес) с показтелем 0.81



## Проверьте модель на тестовой выборке

### Дерево решений

In [15]:
model = DecisionTreeClassifier(max_depth=3, random_state=12345)
model.fit(features_train, target_train)
result = model.score(features_test, target_test)
print("Качество:", result)

Качество: 0.7791601866251944


### Случайный лес

In [16]:
model = RandomForestClassifier(max_depth=8, n_estimators=40, random_state=12345)
model.fit(features_train, target_train)
result = model.score(features_test, target_test)
print("Качество:", result)

Качество: 0.7962674961119751


### Логистическая регрессия

In [17]:
model = LogisticRegression(solver='liblinear')
model.fit(features_train, target_train)
result = model.score(features_test, target_test)
print("Качество:", result)

Качество: 0.7402799377916018


### Вывод

Наибольшая точность модели на тестовой выборке достигается при применении алгоритма RandomForestClassifier (случайный лес) с показтелем 0.793, точность чуть меньше, чем по валидационной выборке
 

## (бонус) Проверьте модели на адекватность

Повторим расчет точности каждой из моделей с помощью функции

In [18]:
def all_models_accuracy(features_train, target_train, features_val, target_val):
    model_DTC = DecisionTreeClassifier(random_state=12345, max_depth=3)
    DTC_score = model_DTC.fit(features_train, target_train).score(features_val, target_val)
    
    model_RFC = RandomForestClassifier(random_state=12345, max_depth=8, n_estimators = 40)
    RFC_score = model_RFC.fit(features_train, target_train).score(features_val, target_val)
    
    model_LgR = LogisticRegression(solver = 'liblinear')
    LgR_score = model_LgR.fit(features_train, target_train).score(features_val, target_val)
    print("Точность:" "- дерево решений", DTC_score)
    print("Точность:" "- случайный лес ", RFC_score)
    print("Точность:" "- логистческая регрессия", LgR_score)
all_models_accuracy(features_train, target_train, features_val, target_val)


Точность:- дерево решений 0.7853810264385692
Точность:- случайный лес  0.8087091757387247
Точность:- логистческая регрессия 0.7589424572317263


Случайный лес показывал самую высокую точность по валидационной выборке - 0.81

<br> Сравним качество предсказаний с новой постоянной моделью. 

In [19]:
#Создадим постаянную модель для проверки адекватности наших моделей, за константу примем 0
target_predict_const = pd.Series([0]*len(target_val))
target_predict_const.shape

(643,)

In [20]:
accuracy_score_const = accuracy_score(target_val, target_predict_const)
accuracy_score_const

0.7060653188180405

Точность постоянной модели = 0.7, что ниже всех оценных нами моделей. Соответственно, все модели являются адекватными.

## Вывод

- В ходе проекта мы разделили данные на 3 выборки в соотношении 3:1:1 (обучающую с 60% данными, валидационную и тестовую по 20% данных)
- Построили 3 модели для задачи классификации и выбора подходящего тарифа для пользователей
- Обучили модели и проверили их точность на валидационной и тестовой выборках
- Наилучший результат показала модель "Случайного леса" с показателем 0.81 по валидачионной выборке
- В заключении подтвердили адекватность всех построенных моделей, сравних их с постоянной моделью

Для выполнения задачи классификации по тарифам и отбора подходящего рекомендуется к применению модель "Случайного леса" с точностью более 80%